In [23]:
import pandas as pd
from collections import Counter  
import json

# Twitter API keys

In [3]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = 'xf96zGbBtmbXnIJBIKbhg7IQ1'
credentials['CONSUMER_SECRET'] = 'HyHYCA8gB1oGN9H5bMEotatc3AfSAMSCK2SeoFVuTdYVwxTpgY'
credentials['ACCESS_TOKEN'] = '106489559-rxeRE7aH8f9Z7V50CsHfaYeZ7DrzmaJBAmn8miNi'
credentials['ACCESS_SECRET'] = 'lJbQlDcLjOtbZNSyJ1qAVJ7jvBpBEJOHnAOoI1KPaetJj'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

### Getting to know the Twitter API requests

In [4]:
# Import the Twython class
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': '#refugees', 
        'result_type': 'popular',
        'count': 20,
        'lang': 'en'
        }

In [5]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'hashtag': [], 'location':[]}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['hashtag'].append(status['entities']['hashtags'])
    dict_['location'].append(status['user']['location'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  

In [30]:
df.shape

(13, 5)

# Using Streamining to obtain more twits using the module "TwythonStreamer" from the library "Twython"

In [6]:
from twython import TwythonStreamer  
import csv

# Filter out unwanted data
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

### Initiating the Streaming from our streaming class until the connection is lost

In [7]:

stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='refugees')

KeyboardInterrupt: 

### Transforming the csv file to a DataFrame

In [4]:
import pandas as pd  
headers = ['hashtags', 'tweet', 'user_name', 'location']
tweets_1 = pd.read_csv("saved_tweets.csv", header = None, names = headers)  
tweets_1.head() 


,hashtags,tweet,user_name,location
0,[],RT @QasimRashid: I’m an American Muslim human ...,pghcovert,the steel city
1,[],"This is a morbid article, but thanks.\n\nInvas...",canucklaw1,"British Columbia, Canada"
2,[],RT @BillKristol: I’ve been wondering why the v...,Miss_de_Haro,NaN
3,[],RT @DVATW: Oh look! Calais gets even more enri...,The_Nitr0,NaN
4,[],RT @QasimRashid: I’m an American Muslim human ...,LunaticEuphoric,"Lahore, Pakistan"


In [6]:
tweets_1.shape

(32475, 4)

### Second Twitter API request with aprox 10K observations

In [7]:
headers = ['hashtags', 'tweet', 'user_name', 'id','location', 'followers']
tweets_2 = pd.read_csv("saved_tweets_refugees_definitive.csv", header = None, names = headers)  


In [8]:
#Dropping the columns 'id' and 'followers' to concatenate it with the table tweets_1

tweets_2 = tweets_2.drop(['id', 'followers'], axis =1)


In [9]:
tweets_2.shape

(9370, 4)

In [10]:
tweets_2.head()

,hashtags,tweet,user_name,location
0,[],RT @charliekirk11: Did you know:\n\nThe govern...,cyelley,"California, USA"
1,[],RT @charliekirk11: Did you know:\n\nThe govern...,metamancer,NaN
2,[],RT @5gBxizlCqzFrc3K: @Refugees @RefugeesChief ...,balkees51046223,Türkiye
3,[],RT @charliekirk11: Did you know:\n\nThe govern...,DoeAnon66,PNW
4,[],RT @charliekirk11: Did you know:\n\nThe govern...,MyPrezTweets,Earth


# Concatenating both dataframes (tweets_1 and tweets_2)

In [11]:
tweets_df = pd.concat([tweets_1,tweets_2])

In [12]:
tweets_df.shape

(41845, 4)

In [367]:
tweets_def.to_csv('twitter_dataset.csv')

#### Removing possible repeated observations

When streaming data from the Twitter API, since we did two streams, Twitter provide us with two datasets. In order to avoid having repeated rows with the same random variables, we decide to eliminate duplicates.

In [13]:
tweets_def = tweets_df.drop_duplicates(subset = ['hashtags', 'tweet', 'user_name', 'location'])

In [14]:
tweets_def.shape

(41647, 4)

# Data wrangling

In [229]:
tweets_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41647 entries, 0 to 9369
Data columns (total 4 columns):
hashtags     41647 non-null object
tweet        41647 non-null object
user_name    41647 non-null object
location     25817 non-null object
dtypes: object(4)
memory usage: 1.6+ MB


##### Check for NaN values

As we can see, there are a lot of Null values in the location column as well a noise in the data with false entries as the users are the ones who write down their respective locations. For this reason we decide to keep them in the main DataFrame as we do not want to drop Null locations which can have values in the Hashtags column. 

### Working with the "location" column:

As mentioned before, the location columns has 15.830 Null values which corresponds to aprox. 36% of out data. For this reason we decided to not drop the null values and just analize the column separately. 

In [230]:
list_location_strings = [entry for entry in tweets_def.location] 

In [231]:
counter_locations = Counter(list_location_strings)

In [232]:
counter_locations.most_common(20)

[(nan, 15830),
 ('United States', 1122),
 ('USA', 401),
 ('California, USA', 354),
 ('Texas, USA', 341),
 ('Florida, USA', 290),
 ('Karabük, Türkiye', 264),
 ('Turkey', 256),
 ('Canada', 192),
 ('Los Angeles, CA', 183),
 ('Mersin, Turkey', 181),
 ('Düzce, Turkey', 154),
 ('North Carolina, USA', 152),
 ('London, England', 146),
 ('Washington, DC', 139),
 ('New York, USA', 134),
 ('Chicago, IL', 123),
 ('🇺🇸', 122),
 ('New York, NY', 114),
 ('Arizona, USA', 113)]

In [233]:
df_location = pd.DataFrame.from_dict(counter_locations, orient='index').reset_index()
df_location = df_location.rename(columns={'index':'locations', 0:'count'})
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9863 entries, 0 to 9862
Data columns (total 2 columns):
locations    9862 non-null object
count        9863 non-null int64
dtypes: int64(1), object(1)
memory usage: 154.2+ KB


In [234]:
df_location = df_location.sort_values(['count'], ascending=False)

In [235]:
df_location = df_location.dropna(axis=0)
df_location.shape

(9862, 2)

In [236]:
df_location.describe()

,count
count,9862.000000
mean,2.617826
std,15.804257
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1122.000000


In [237]:
df_loc_filter = df_location[df_location['count'] > 20]

In [238]:
df_loc_filter.describe()

,count
count,129.000000
mean,78.457364
std,114.412972
min,21.000000
25%,29.000000
50%,49.000000
75%,85.000000
max,1122.000000


In [239]:
df_loc_filter.head()

,locations,count
17,United States,1122
24,USA,401
91,"California, USA",354
98,"Texas, USA",341
111,"Florida, USA",290


In [360]:
df_loc.head()

,locations,count,fixed_locations
24,USA,401,usa
91,"California, USA",354,"california, usa"
98,"Texas, USA",341,"texas, usa"
111,"Florida, USA",290,"florida, usa"
905,"Karabük, Türkiye",264,turkey


In [244]:
def lower_locs(loc):
    return loc.lower()

In [245]:
df_loc['fixed_locations'] = df_loc['locations'].apply(lower_locs)

In [361]:
df_loc.head()

,locations,count,fixed_locations
24,USA,401,usa
91,"California, USA",354,"california, usa"
98,"Texas, USA",341,"texas, usa"
111,"Florida, USA",290,"florida, usa"
905,"Karabük, Türkiye",264,turkey


In [247]:
def fix_names(loc):
    
    if 'turkey' in loc:
        return 'turkey'
    
    elif 'türkiye' in loc:
        return 'turkey'
    
    elif 'sakarya' in loc:
        return 'turkey'
    
    elif 'deutschland' in loc:
        return 'deutschland'
    
    elif 'england' in loc:
        return 'united kingdom'
    
    elif 'london' in loc:
        return 'united kingdom'
    
    elif 'united kingdom' in loc:
        return 'united kingdom'
    
    elif 'british columbia' in loc:
        return 'canada'
    
    elif 'ontario' in loc:
        return 'canada'
    
    elif 'calgary' in loc:
        return 'canada'
    
    return loc
    

In [254]:
df_loc['fixed_locations'] = df_loc['fixed_locations'].apply(fix_names)

In [364]:
df_loc_def = df_loc

In [365]:
df_loc_def = df_loc_def.drop('locations', axis =1)

In [366]:
df_loc_def.head()

,count,fixed_locations
24,401,usa
91,354,"california, usa"
98,341,"texas, usa"
111,290,"florida, usa"
905,264,turkey


#### Obtaining a json file with the states of the United States

In [332]:
url = 'https://gist.github.com/mshafrir/2646763#file-states_hash-json'

In [333]:
import json

with open('states_hash.json') as json_file:  
    data = json.load(json_file)

In [334]:
us_states = [state.lower() for state in data.values()]

In [338]:
def fixed_usa(loc):
    for i in range(len(us_states)):
        if us_states[i].lower() in loc:
            return us_states[i].lower()
    return loc
    

In [339]:
df_loc_def['fixed_locations'] = df_loc_def['fixed_locations'].apply(fixed_usa)

In [363]:
df_loc_def.head()

,count,fixed_locations,fixed_locations_2
24,401,usa,None
91,354,california,california
98,341,texas,texas
111,290,florida,florida
905,264,turkey,turkey


##### Eliminating locations that are not in my countries list or in my american states states list

In [341]:
countries = ['turkey','deutschland','united kingdom','canada','germany']
    
def eliminating_locations(loc):
    if (loc in countries):
        return loc
    elif loc in us_states:
        return loc
    

In [342]:
df_loc_def['fixed_locations_2'] = df_loc_def['fixed_locations']

In [343]:
df_loc_def.head()

,count,fixed_locations,fixed_locations_2
24,401,usa,usa
91,354,california,california
98,341,texas,texas
111,290,florida,florida
905,264,turkey,turkey


In [344]:
df_loc_def['fixed_locations_2'] = df_loc_def['fixed_locations_2'].map(eliminating_locations)

In [351]:
df_locations_definitive = df_loc_def.dropna().drop('fixed_locations', axis = 1)

In [356]:
df_locations_definitive.columns = ['counts', 'locations']

In [358]:
df_locations_definitive.groupby('locations').sum()

,counts
locations,
alabama,64
arizona,142
california,486
canada,389
colorado,101
connecticut,32
deutschland,25
florida,358
georgia,91


## Getting most popular hashtags

In [59]:
from collections import Counter  
import ast

#tweets = pd.read_csv("saved_tweets.csv")

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets_def.hashtags]  
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))  
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)  
counter_hashtags.most_common(50) 

[('refugees', 301),
 ('concentrationcamps', 128),
 ('libya', 113),
 ('standwithiraqirefugees', 98),
 ('ice', 98),
 ('iceraids', 89),
 ('migrants', 84),
 ('fakechristian', 79),
 ('delaram', 68),
 ('rohingya', 59),
 ('worldrefugeeday', 54),
 ('usa', 52),
 ('iranianrefugeesinturkey', 52),
 ('trump', 48),
 ('forrefugees', 48),
 ('internationalsocialforum2019', 47),
 ('racistpresident', 45),
 ('eu', 43),
 ('saveali', 43),
 ('closethecamps', 35),
 ('standwithlraqirefugees', 34),
 ('turkey', 33),
 ('racistinchief', 32),
 ('resettlement4iranian', 30),
 ('iran', 27),
 ('humanrights', 27),
 ('canada', 27),
 ('peffer', 27),
 ('midwest', 27),
 ('germany', 26),
 ('china', 25),
 ('christians', 25),
 ('evangelicals', 25),
 ('jesus', 25),
 ('psychotherapie', 25),
 ('pkv', 25),
 ('syria', 24),
 ('abolishice', 22),
 ('individual1', 22),
 ('muellerreport', 22),
 ('immigrants', 21),
 ('politics', 21),
 ('france', 19),
 ('sar', 19),
 ('repeatinghistory', 19),
 ('thingsjesusneversaid', 18),
 ('us', 18),
 ('

In [28]:
type(counter_hashtags)

collections.Counter

In [107]:
df_hashtags = pd.DataFrame.from_dict(counter_hashtags, orient='index').reset_index()
df_hashtags = df_hashtags.rename(columns={'index':'hashtags', 0:'count'})
df_hashtags.head()

,hashtags,count
0,crimesagainsthumanity,3
1,crimesagainstchildren,1
2,humanrightsviolations,2
3,china,25
4,tajoura,9


In [108]:
df_hashtags = df_hashtags.sort_values(['count'], ascending=False)

In [112]:
df_hashtags.describe()

,count
count,23.000000
mean,73.173913
std,56.605214
min,32.000000
25%,44.000000
50%,52.000000
75%,86.500000
max,301.000000


In [113]:
df_hashtags = df_hashtags[df_hashtags['count'] > 52]

In [116]:
df_hashtags.reset_index(drop=True)

,hashtags,count
0,refugees,301
1,concentrationcamps,128
2,libya,113
3,standwithiraqirefugees,98
4,ice,98
5,iceraids,89
6,migrants,84
7,fakechristian,79
8,delaram,68
9,rohingya,59


## Get the most popular tweet

In [119]:
tweets_def.head()

,hashtags,tweet,user_name,location
0,[],RT @QasimRashid: I’m an American Muslim human ...,pghcovert,the steel city
1,[],"This is a morbid article, but thanks.\n\nInvas...",canucklaw1,"British Columbia, Canada"
2,[],RT @BillKristol: I’ve been wondering why the v...,Miss_de_Haro,NaN
3,[],RT @DVATW: Oh look! Calais gets even more enri...,The_Nitr0,NaN
4,[],RT @QasimRashid: I’m an American Muslim human ...,LunaticEuphoric,"Lahore, Pakistan"


In [128]:
popular_tweets = tweets_def.groupby('tweet').count().sort_values(['hashtags'], ascending=False)

In [131]:
popular_tweets.head()

,hashtags,user_name,location
tweet,,,
"RT @charliekirk11: Did you know:\n\nThe government pays out $2,125/month in refugee benefits to refugees resettled in the United States\n\nMean…",4434,4434,2552
"RT @BillKristol: I’ve been wondering why the video of the VP and senators at the border is so disconcerting. Is it that they enter, gaze at…",1942,1942,1244
RT @_SJPeace_: @realDonaldTrump Most refugees from the Middle East would love to go back to their country but american bombs have destroyed…,1658,1658,1097
RT @Speech7x7: 🇳🇱Netherlands : if you get Attacked by Muslim refugees by bricks or eggs.😕do not complain to police.❗️They might just throw-…,1428,1428,944
RT @QasimRashid: I’m an American Muslim human rights lawyer running for VA Senate &amp; I’m being reported for helping desperate Christian refu…,991,991,635


In [132]:
popular_tweets = popular_tweets.drop(['user_name', 'location'], axis=1)

In [135]:
popular_tweets.columns = ['count']

In [136]:
popular_tweets.head()

,count
tweet,
"RT @charliekirk11: Did you know:\n\nThe government pays out $2,125/month in refugee benefits to refugees resettled in the United States\n\nMean…",4434
"RT @BillKristol: I’ve been wondering why the video of the VP and senators at the border is so disconcerting. Is it that they enter, gaze at…",1942
RT @_SJPeace_: @realDonaldTrump Most refugees from the Middle East would love to go back to their country but american bombs have destroyed…,1658
RT @Speech7x7: 🇳🇱Netherlands : if you get Attacked by Muslim refugees by bricks or eggs.😕do not complain to police.❗️They might just throw-…,1428
RT @QasimRashid: I’m an American Muslim human rights lawyer running for VA Senate &amp; I’m being reported for helping desperate Christian refu…,991


In [140]:
lst_index = popular_tweets.index.to_numpy()

In [142]:
most_popular_twit = lst_index[0]

In [143]:
most_popular_twit

'RT @charliekirk11: Did you know:\n\nThe government pays out $2,125/month in refugee benefits to refugees resettled in the United States\n\nMean…'